### The data sets are from Kaggle competition: https://www.kaggle.com/c/digit-recognizer/data?select=test.csv

In [ ]:
import pandas as pd
import numpy as np

train = np.asarray(pd.read_csv('train.csv'))
test = np.asarray(pd.read_csv('test.csv'))

In [ ]:
# Convert X to 28 x 28 and y to a vector
X_train = train[:, 1:].reshape(-1, 28, 28)
y_train = train[:,0].reshape(-1,)
X_test = test.reshape(-1, 28, 28)

In [ ]:
# Show an image example
import matplotlib.pyplot as plt
%matplotlib inline
plt.gray()
plt.matshow(X_train[0])

In [ ]:
# Split the data sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.1)

In [ ]:
# One-hot encode y_train and y_test
# from tensorflow.keras.utils import to_categorical

# y_train = to_categorical(y_train, 10).astype(int)
# y_val = to_categorical(y_val, 10).astype(int)

In [ ]:
# # Normalize X_train and X_val
# X_train = X_train.astype(float) / 255.0
# X_val = X_val.astype(float) / 255.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

input_shape = (X_train.shape[1], )
# Create the keras tuner model.
def build_model(learning_rate = 0.01, activation = tf.nn.leaky_relu):
    model = Sequential()
    model.add(Dense(32, activation=tf.nn.leaky_relu))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation=tf.nn.leaky_relu))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation=tf.nn.leaky_relu))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

model = KerasClassifier(build_fn = build_model,
                       verbose = 0)
# Define the parameters to try out
params = {'batch_size': [16, 32, 64], 
          'epochs': [50, 100],
          'learning_rate': [0.01, 0.001, 0.0001]}

random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = StratifiedKFold(10))

In [9]:
random_search_results = random_search.fit(X_train, y_train, verbose=1)

2127/2127 [==============================] - 7s 3ms/step - loss: 0.1113 - accuracy: 0.9649
Epoch 28/50
2127/2127 [==============================] - 7s 3ms/step - loss: 0.1150 - accuracy: 0.9617
Epoch 29/50
2127/2127 [==============================] - 7s 3ms/step - loss: 0.1101 - accuracy: 0.9654
Epoch 30/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1124 - accuracy: 0.9657
Epoch 31/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1126 - accuracy: 0.9628
Epoch 32/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1079 - accuracy: 0.9656
Epoch 33/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1103 - accuracy: 0.9644
Epoch 34/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1090 - accuracy: 0.9652
Epoch 35/50
2127/2127 [==============================] - 6s 3ms/step - loss: 0.1095 - accuracy: 0.9648
Epoch 36/50
2127/2127 [==============================] - 7s 3ms/step - loss: 0.1013 -

In [10]:
# Predict target vector
y_pred = random_search_results.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=['Label'])
y_pred.index += 1
y_pred.index.name='ImageId'
y_pred.to_csv('prediction.csv', index=True)

C:\Users\dtngh\miniconda3\envs\gputest\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [11]:
import os
os.system("shutdown /s")

0